Importing the libraries

In [1]:
import numpy as np
from scipy.integrate import solve_ivp,odeint
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

Defining Constants and reference quantities to make all further calculations dimensionless
Here K1 and K2 are constants taken to make all quantities dimensionless
Note : All the quanties in this project are taken in cgs units

In [ ]:
#Define universal gravitation constant
G = 6.67408e-11 #N-m2/kg2
#Reference quantities
m_nd = 1.989e+30
r_nd = 147.1e+9
v_nd = 29800
t_nd = 365.2*24*3600
# kg
# m
# m/s
# s
#mass of the sun
#distance between sun and earth
#relative velocity of earth around the sun
#orbital period of Earth
#Net constants
K1 = G*t_nd*m_nd/(r_nd**2*v_nd)
K2 = v_nd*t_nd/r_nd

Now defining masses and velocties of all the bodies present in solar system and their distance from
the Sun
All the data are taken from the NASA website (https://nssdc.gsfc.nasa.gov/planetary/factsheet/)

In [ ]:
# Define masses
m1 = 1
m2 = 0.33e+24/m_nd
m3 = 4.87e+24/m_nd
m4 = 5.97e+24/m_nd
m5 = 0.642e+24/m_nd
# Sun
# Mercury
# Venus
# Earth
# Mars

m6 = 1898e+24/m_nd
m7 = 568e+24/m_nd
m8 = 86.8e+24/m_nd
m9 = 102e+24/m_nd
# Jupiter
# Saturn
# Uranus
# Neptune
masses = np.array([m1,m2,m3,m4,m5,m6,m7,m8,m9])
#Define initial position vectors
r1 = np.array([0,0,0]) # Sun
r2 = np.array([57.9e+9/r_nd,0,0])
r3 = np.array([108.2e+9/r_nd,0,0])
r4 = np.array([1,0,0]) # Earth
r5 = np.array([228e+9/r_nd,0,0])
r6 = np.array([778.5e+9/r_nd,0,0])
r7 = np.array([1432e+9/r_nd,0,0])
r8 = np.array([2867e+9/r_nd,0,0])
r9 = np.array([4515e+9/r_nd,0,0])
#Define initial velocities
v1 = np.array([0,0,0])
v2 = np.array([0,47.4e+3/v_nd,0])
v3 = np.array([0,35e+3/v_nd,0])
v4 = np.array([0,1,0]) #m/s
v5 = np.array([0,24.1e+3/v_nd,0])
v6 = np.array([0,13.1e+3/v_nd,0])
v7 = np.array([0,9.7e+3/v_nd,0])
v8 = np.array([0,6.8e+3/v_nd,0])
v9 = np.array([0,5.4e+3/v_nd,0])
#Find Position of Centre of Mass
r_com=(m1*r1+m2*r2+m3*r3+m4*r4+m5*r5+m6*r6+m7*r7+m8*r8+m9*r9)/(m1+m2+m3+m4+m5+m6+m7+m8+m9)
#Find velocity of COM
v_com=(m1*v1+m2*v2+m3*v3+m4*v4+m5*v5+m6*v6+m7*v7+m8*v8+m9*v9)/(m1+m2+m3+m4+m5+m6+m7+m8+m9)
print("Position of center of mass of Solar System : ",r_com, "AU")
print("Velocity of center of mass of Solar System : ",v_com*v_nd, "m/s")

Now Defining a function which takes all these quantities of planetary bodies and provide the
deriveties as an output to be used fro intergration
Steps involved in this are as following:-
First check number of masses which will provide the number of bodies provide for the calculation
Then make 2D array of distances (r) of all these bodies with dimension = 𝑛𝑢𝑚𝑏𝑒𝑟 𝑜𝑓 𝑏𝑜𝑑𝑖𝑒𝑠 × 3
After that a 3D array is created to find the distance between these bodies with each other Later this
3D array of differene is norm that can be used the in the formula to calulate the net acceleration
on each body due to gravitational force from the othe bodies
The formula used here to calulate the net acceleration here is -
𝑖
𝑚𝑖 𝑑𝑣
𝑑𝑡 = 𝐾1 ∑𝑗
𝑚𝑖 𝑚𝑗
𝑟𝑖𝑗
3
𝑟𝑖𝑗
and for net velocity -
𝑑𝑥𝑖
𝑑𝑡 = 𝐾2 𝑣𝑖
Then finally all the derivatives are flatten so the ode solver have 1 dimension input array only.

In [ ]:
def NBodyEquations(t,w,G,masses):
    num_bodies = len(masses)
    r = w[:num_bodies*3].reshape((num_bodies,3))
    v = w[num_bodies*3:].reshape((num_bodies,3))
    r_diff = r[np.newaxis, :, :] - r[:, np.newaxis, :]
    r_norm = np.linalg.norm(r_diff,axis=2)
    r_norm_cube = r_norm**3
    v_derivs = np.zeros([num_bodies,3])
    for row in range(num_bodies):
        for col in range(num_bodies):
            if col!=row:
                v_derivs[row] += masses[col]*r_diff[row,col,:]/r_norm_cube[row,col]
    v_derivs = K1*v_derivs
    r_derivs = K2*v
    derivs = np.concatenate((r_derivs.flatten(),v_derivs.flatten()))
    return derivs

Defining input parameters and time span for the ode to solve
Here 100 orbital periods are taken

In [ ]:
init_params=np.array([r1,r2,r3,r4,r5,r6,r7,r8,r9,v1,v2,v3,v4,v5,v6,v7,v8,v9]) #create array of initial params
init_params=init_params.flatten() #flatten array to make it 1D
time_span = [0, 100] # 100 orbitals = 100 years
t_eval = np.linspace(time_span[0], time_span[1], 5000) # step = 100*365.2/5000= 7.304 days
N_body_sol = solve_ivp(NBodyEquations, time_span, init_params, args=(G,masses),rtol=1e-10,atol=1e-15,t_eval=t_eval)
R_sol = N_body_sol.y # taking the values of postions of bodies
print("Duration for which the solver evaluate the equaition = ",time_span[1],"years")
print("One step in solver = ",time_span[1]*356.2/5000, "days")

Ploting the 3D figure to see the positions of the bodies

In [ ]:
#Create figure
fig=plt.figure(figsize=(15,15))
#Create 3D axes
ax=fig.add_subplot(111,projection="3d")
Colors = ["yellow","tab:green","tab:purple","tab:blue","tab:red","tab:orange","tab:pink","tab:cyan","tab:brown","tab:gray"]
Label =["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune","Star"]
#Plot the orbits
for i in range(len(masses)):
    ax.plot(R_sol[3*i],R_sol[3*i +1],R_sol[3*i+2],color = Colors[i])
    ax.scatter(R_sol[3*i][-1],R_sol[3*i +1][-1],R_sol[3*i+2][-1],color =Colors[i],marker="o",s=100,label=Label[i])
ax.set_xlabel("x-coordinate",fontsize=14)
ax.set_ylabel("y-coordinate",fontsize=14)
ax.set_zlabel("z-coordinate",fontsize=14)
ax.set_title("Visualization of orbits of planets in the solar system\n",fontsize=14)
ax.legend(loc="upper left",fontsize=14)